In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torchsummary import summary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
if os.getcwd() == '/content':
    # os.chdir('/content/drive/Shareddrives/시게열 모델링/data')
    os.chdir('/content/drive/MyDrive/time series/jw')

In [ ]:
act = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-01.온라인 행동 정보.csv')
demo = pd.read_csv('제6회 L.POINT Big Data Competition-분석용데이터-03.고객 Demographic 정보.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
act = act.sort_values(by=['clnt_id', 'sess_id', 'hit_seq'])
act

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
3102890,1,1,1,0,A01,20190911,16:14,11880,NaN,과일선물세트,14.0,124.0,unknown,mobile_app
3136925,1,1,2,0,A01,20190911,16:15,22432,NaN,과일선물세트 백화점,14.0,124.0,unknown,mobile_app
3065714,1,1,3,0,A01,20190911,16:15,36140,NaN,과일바구니,14.0,124.0,unknown,mobile_app
3091872,1,2,1,0,A01,20190922,14:09,41584,NaN,초등가을잠바,45.0,424.0,unknown,mobile_app
3142610,1,2,2,0,A01,20190922,14:10,56113,NaN,초등가을점퍼,45.0,424.0,unknown,mobile_app
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148589,72428,3,3,0,A01,20190914,21:50,839771,NaN,페레가모가방,113.0,1295.0,unknown,mobile_app
3148588,72428,4,1,0,A01,20190914,23:42,243555,NaN,펜디가방,35.0,418.0,unknown,mobile_app
3158259,72428,5,1,0,A01,20190918,06:37,12568,NaN,라인에디션블라우스,14.0,24.0,unknown,mobile_app
1759477,72429,1,1,1,A03,20190919,22:09,839064,NaN,NaN,9.0,984.0,DIRECT,mobile_web


In [ ]:
act['hit_pss_tm'] = act.groupby('clnt_id')['hit_pss_tm'].diff().replace({np.nan: 0})
act

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm
3102890,1,1,1,0,A01,20190911,16:14,0.0,NaN,과일선물세트,14.0,124.0,unknown,mobile_app
3136925,1,1,2,0,A01,20190911,16:15,10552.0,NaN,과일선물세트 백화점,14.0,124.0,unknown,mobile_app
3065714,1,1,3,0,A01,20190911,16:15,13708.0,NaN,과일바구니,14.0,124.0,unknown,mobile_app
3091872,1,2,1,0,A01,20190922,14:09,5444.0,NaN,초등가을잠바,45.0,424.0,unknown,mobile_app
3142610,1,2,2,0,A01,20190922,14:10,14529.0,NaN,초등가을점퍼,45.0,424.0,unknown,mobile_app
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148589,72428,3,3,0,A01,20190914,21:50,215034.0,NaN,페레가모가방,113.0,1295.0,unknown,mobile_app
3148588,72428,4,1,0,A01,20190914,23:42,-596216.0,NaN,펜디가방,35.0,418.0,unknown,mobile_app
3158259,72428,5,1,0,A01,20190918,06:37,-230987.0,NaN,라인에디션블라우스,14.0,24.0,unknown,mobile_app
1759477,72429,1,1,1,A03,20190919,22:09,0.0,NaN,NaN,9.0,984.0,DIRECT,mobile_web


In [ ]:
demo

,clnt_id,clnt_gender,clnt_age
0,1,unknown,unknown
1,2,F,30
2,3,unknown,unknown
3,4,unknown,unknown
4,5,unknown,unknown
...,...,...,...
72394,72425,unknown,unknown
72395,72426,unknown,unknown
72396,72427,unknown,unknown
72397,72428,unknown,unknown


In [ ]:
act = act.merge(demo, left_on='clnt_id', right_on='clnt_id', how='left')
act

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age
0,1,1,1,0,A01,20190911,16:14,0.0,NaN,과일선물세트,14.0,124.0,unknown,mobile_app,unknown,unknown
1,1,1,2,0,A01,20190911,16:15,10552.0,NaN,과일선물세트 백화점,14.0,124.0,unknown,mobile_app,unknown,unknown
2,1,1,3,0,A01,20190911,16:15,13708.0,NaN,과일바구니,14.0,124.0,unknown,mobile_app,unknown,unknown
3,1,2,1,0,A01,20190922,14:09,5444.0,NaN,초등가을잠바,45.0,424.0,unknown,mobile_app,unknown,unknown
4,1,2,2,0,A01,20190922,14:10,14529.0,NaN,초등가을점퍼,45.0,424.0,unknown,mobile_app,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,72428,3,3,0,A01,20190914,21:50,215034.0,NaN,페레가모가방,113.0,1295.0,unknown,mobile_app,unknown,unknown
3196358,72428,4,1,0,A01,20190914,23:42,-596216.0,NaN,펜디가방,35.0,418.0,unknown,mobile_app,unknown,unknown
3196359,72428,5,1,0,A01,20190918,06:37,-230987.0,NaN,라인에디션블라우스,14.0,24.0,unknown,mobile_app,unknown,unknown
3196360,72429,1,1,1,A03,20190919,22:09,0.0,NaN,NaN,9.0,984.0,DIRECT,mobile_web,unknown,unknown


In [ ]:
cat_features = ['action_type', 'biz_unit', 'trfc_src', 'dvc_ctg_nm', 'clnt_gender', 'clnt_age']
numeric_features = ['hit_pss_tm', 'tot_pag_view_ct', 'tot_sess_hr_v']

In [ ]:
# tokenizing
token = {
'action_type': {'unknown': 0,
                '검색': 1,
                '제품 목록': 2, 
                '제품 세부정보': 3,
                '장바구니 추가': 4,
                '장바구니 삭제': 5,
                '결제 시도': 6,
                '구매 완료': 7,
                '구매 환불': 8},
'biz_unit': {'unknown': 0,
             'A01': 1, 
             'A02': 2, 
             'A03': 3},
'trfc_src': {'unknown': 0, 
             'DIRECT': 1, 
             'PUSH': 2, 
             'PORTAL_1': 3, 
             'WEBSITE': 4, 
             'PORTAL_2': 5,
             'PORTAL_3': 6},
'dvc_ctg_nm': {'unknown': 0,
               'mobile_app': 1,
               'mobile_web': 2,
               'PC': 3},
'clnt_gender': {'unknown': 0,
                'F': 1,
                'M': 2},
'clnt_age': {'unknown': 0,
             '10': 1,
             '20': 2,
             '30': 3,
             '40': 4,
             '50': 5,
             '60': 6,}
}

In [ ]:
act['action_type'] = act['action_type'] + 1 # action_type 0: padding 
act['biz_unit'] = act['biz_unit'].replace(token['biz_unit'])
act['trfc_src'] = act['trfc_src'].replace(token['trfc_src'])
act['dvc_ctg_nm'] = act['dvc_ctg_nm'].replace(np.nan, 'unknown').replace(token['dvc_ctg_nm'])
act['clnt_gender'] = act['clnt_gender'].replace(token['clnt_gender'])
act['clnt_age'] = act['clnt_age'].replace(token['clnt_age'])

In [ ]:
act

,clnt_id,sess_id,hit_seq,action_type,biz_unit,sess_dt,hit_tm,hit_pss_tm,trans_id,sech_kwd,tot_pag_view_ct,tot_sess_hr_v,trfc_src,dvc_ctg_nm,clnt_gender,clnt_age
0,1,1,1,1,1,20190911,16:14,0.0,NaN,과일선물세트,14.0,124.0,0,1,0,0
1,1,1,2,1,1,20190911,16:15,10552.0,NaN,과일선물세트 백화점,14.0,124.0,0,1,0,0
2,1,1,3,1,1,20190911,16:15,13708.0,NaN,과일바구니,14.0,124.0,0,1,0,0
3,1,2,1,1,1,20190922,14:09,5444.0,NaN,초등가을잠바,45.0,424.0,0,1,0,0
4,1,2,2,1,1,20190922,14:10,14529.0,NaN,초등가을점퍼,45.0,424.0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3196357,72428,3,3,1,1,20190914,21:50,215034.0,NaN,페레가모가방,113.0,1295.0,0,1,0,0
3196358,72428,4,1,1,1,20190914,23:42,-596216.0,NaN,펜디가방,35.0,418.0,0,1,0,0
3196359,72428,5,1,1,1,20190918,06:37,-230987.0,NaN,라인에디션블라우스,14.0,24.0,0,1,0,0
3196360,72429,1,1,2,3,20190919,22:09,0.0,NaN,NaN,9.0,984.0,1,2,0,0


In [ ]:
for col in cat_features:
    print(act[col].unique())

[1 2 3 4 6 7 5 8]
[1 3 2]
[0 1 2 3 4 5 6]
[1 0 2 3]
[0 1 2]
[0 3 4 2 5 6 1]


In [ ]:
[act[col].unique().shape[0] for col in cat_features]

[8, 3, 7, 4, 3, 7]

In [ ]:
# 'action_type', 'hit_seq', 'biz_unit' 는 패딩idx 0의 값이 없지만, 
# 있을거라고 가정해서 범주 개수를 1개 늘려줘야함
n_classes = [9, 4, 7, 4, 3, 7]

In [ ]:
# n_times 기간의 action_type 시퀀스를 이용해
# 다음 predict_period 스텝의 action_type을 예측하는 task로
# 임베딩 레이어를 fitting 한다
# 따라서 최소 n_times + predict_period 개의 데이터가 있어야 함

n_times = 10
predict_period = 1

sliding window trick: 미리 인덱스 matrix를 만들어서 한번에 인덱싱하자!
for 문으로 루프 돌리니 개느림
  
https://towardsdatascience.com/fast-and-robust-sliding-window-vectorization-with-numpy-3ad950ed62f5


In [ ]:
X, y = [], []
def make_seq(x, target_col='action_type'):
    # 최소 n_times + predict_period 개의 데이터가 있는 것에 한해서,
    if len(x) >= n_times + predict_period:
        # with sliding window trick
        I_src = np.arange(n_times).reshape(1, -1)
        S_src = np.arange(len(x)-n_times-predict_period+1).reshape(-1, 1)
        src = x.values[I_src+S_src]

        I_trg = np.arange(predict_period).reshape(1, -1)
        S_trg = np.arange(n_times, len(x)).reshape(-1, 1)
        trg = x[target_col].values[I_trg+S_trg]

        X.append(src)
        y.append(trg)

In [ ]:
# sess_id 별로 시퀀스 생성 -> 10초만에 끝!
(act[['clnt_id', 'sess_id']+cat_features+numeric_features]
 .groupby(['clnt_id', 'sess_id'])
 .apply(lambda x: make_seq(x)))

""


In [ ]:
X = np.concatenate(X, axis=0)[:, :, 2:] # index 0,1 of last dim is clnt_id, sess_id
y = np.concatenate(y, axis=0)
X.shape, y.shape

((1549550, 10, 9), (1549550, 1))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.7, 
                                                    random_state=96,
                                                    stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, 
                                                train_size=0.5, 
                                                random_state=96,
                                                stratify=y_test)
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((1084685, 10, 9),
 (232432, 10, 9),
 (232433, 10, 9),
 (1084685, 1),
 (232432, 1),
 (232433, 1))

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
output_dim = n_classes[0]
embedding_dim = 64
hidden_size = 128
num_layers = 3
num_nodes = [1024] * 5
dropout = 0.3

In [ ]:
class Act2Vec(nn.Module):
    def __init__(self, 
                 n_classes=n_classes, 
                 n_times=n_times,
                 output_dim=output_dim, 
                 embedding_dim=embedding_dim, 
                 hidden_size=hidden_size, 
                 num_layers=num_layers,
                 num_nodes=num_nodes,
                 dropout=dropout):
        super(Act2Vec, self).__init__()

        self.n_classes = n_classes
        self.n_times = n_times
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_dim = output_dim
        self.num_nodes = num_nodes
        self.activation = nn.SiLU()
        self.dropout = dropout
        self.dropout_layer = nn.Dropout(self.dropout)

        self.embed_layers = nn.ModuleList([nn.Embedding(num_embeddings=n_class,
                                                        embedding_dim=self.embedding_dim,
                                                        padding_idx=0) 
                                            for n_class in self.n_classes])
        self.lstm = nn.LSTM(input_size=len(self.n_classes)*self.embedding_dim, 
                            hidden_size=self.hidden_size, 
                            num_layers=self.num_layers,
                            batch_first=True,
                            dropout=self.dropout)
        self.flatten = nn.Flatten()

        self.prev_num_nodes = [self.n_times*self.hidden_size] + self.num_nodes[:-1]
        self.dense_layers = nn.ModuleList([nn.Linear(n1, n2) 
                                            for n1, n2 in zip(self.prev_num_nodes, 
                                                              self.num_nodes)])
        self.output_layer = nn.Linear(self.num_nodes[-1], 
                                      self.output_dim)

    def forward(self, x):
        # embedding 레이어는 정수 텐서 입력을 받음
        # Dataset에서 처리해줄거긴 한데 혹시 모르니 안전빵...
        x = x.type(torch.LongTensor).to(device)

        # output of ith feature's embedding layer.
        embeddeds = [layer(x[:, :, i]) for i, layer in enumerate(self.embed_layers)]
        x = torch.concat(embeddeds, dim=-1)

        # lstm forward
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        x, _ = self.lstm(x, (h0, c0))   # x: (batch, seq_len, hidden_size)

        x = self.flatten(x)
        x = self.dropout_layer(x)

        # Dense layer forward
        for layer in self.dense_layers:
            x = layer(x)
            x = self.activation(x)
            x = self.dropout_layer(x)

        # output
        out = self.output_layer(x)

        return out

    def embed(self, x):
        return [layer(x[:, :, i]) for i, layer in enumerate(self.embed_layers)]

In [ ]:
class ActDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.LongTensor(x)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

In [ ]:
train_ds = ActDataset(X_train, y_train)
val_ds = ActDataset(X_val, y_val)
test_ds = ActDataset(X_test, y_test)

In [ ]:
BATCH_SIZE = 32768
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE)
val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [ ]:
learning_rate = 0.001
momentum = 0.9
num_epochs = 50
smoothing = 0.0
path = './Act2Vec'

act2vec = Act2Vec().to(device)
criterion = nn.CrossEntropyLoss(label_smoothing=smoothing)
optimizer = optim.Adam(act2vec.parameters(), lr=learning_rate)
# base_optimizer = optim.Adam
# optimizer = SAM(act2vec.parameters(), base_optimizer, lr=learning_rate)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size = 5,
#                                                gamma = 0.8)
act2vec

Act2Vec(
  (activation): SiLU()
  (dropout_layer): Dropout(p=0.3, inplace=False)
  (embed_layers): ModuleList(
    (0): Embedding(9, 64, padding_idx=0)
    (1): Embedding(4, 64, padding_idx=0)
    (2): Embedding(7, 64, padding_idx=0)
    (3): Embedding(4, 64, padding_idx=0)
    (4): Embedding(3, 64, padding_idx=0)
    (5): Embedding(7, 64, padding_idx=0)
  )
  (lstm): LSTM(384, 128, num_layers=3, batch_first=True, dropout=0.3)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dense_layers): ModuleList(
    (0): Linear(in_features=1280, out_features=1024, bias=True)
    (1): Linear(in_features=1024, out_features=1024, bias=True)
    (2): Linear(in_features=1024, out_features=1024, bias=True)
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (output_layer): Linear(in_features=1024, out_features=9, bias=True)
)

In [ ]:
# Train the model
total_step = len(train_dl)
for epoch in range(num_epochs):
    train_loss = 0.0
    correct = 0
    total = 0
    act2vec.train()

    # train
    tqdm_loader = tqdm(train_dl)    # for print loss while epoch progressing
    for i, (data, labels) in enumerate(tqdm_loader):
        data = data.to(device)
        labels = torch.squeeze(labels).to(device)
        
        with torch.set_grad_enabled(True):
            # Forward pass
            outputs = act2vec(data)
            loss = criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()

            # optimizer.first_step(zero_grad=True)
            optimizer.step()
            criterion(act2vec(data), labels).backward()
            # optimizer.second_step(zero_grad=True)

        # accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # print informations
        tqdm_loader.set_description('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc:{:.4f}' 
                                    .format(epoch+1, 
                                            num_epochs, 
                                            i+1, 
                                            total_step, 
                                            loss.item(),
                                            correct/total))
        train_loss += loss.item()*data.size(0)

    if (epoch+1) % 5 == 0:
        torch.save({
            'epoch': epoch,
            'optimizer_state_dict': optimizer.state_dict(),
            'model_state_dict': act2vec.state_dict()}, 
            f'{path}/epoch{epoch+1:04d}.pth')
        print('Checkpoint saved successfully.')

    # validation
    act2vec.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, labels in val_dl:
            data = data.to(device)
            labels = torch.squeeze(labels).to(device)

            outputs = act2vec(data)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Validation Accuracy: {:.4f}'.format(correct / total)) 

Epoch [1/50], Step [34/34], Loss: 1.7182, Acc:0.2633: 100%|██████████| 34/34 [00:36<00:00,  1.09s/it]


Validation Accuracy: 0.3453


Epoch [2/50], Step [34/34], Loss: 1.4660, Acc:0.3786: 100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


Validation Accuracy: 0.4637


Epoch [3/50], Step [34/34], Loss: 1.2305, Acc:0.5174: 100%|██████████| 34/34 [00:36<00:00,  1.08s/it]


...

Validation Accuracy: 0.6206


Epoch [49/50], Step [34/34], Loss: 0.9749, Acc:0.6193: 100%|██████████| 34/34 [00:36<00:00,  1.09s/it]


Validation Accuracy: 0.6215


Epoch [50/50], Step [34/34], Loss: 0.9770, Acc:0.6196: 100%|██████████| 34/34 [00:36<00:00,  1.09s/it]


Checkpoint saved successfully.
Validation Accuracy: 0.6213


In [ ]:
# test
act2vec.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in test_dl:
        data = data.to(device)
        labels = torch.squeeze(labels).to(device)

        outputs = act2vec(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('test Accuracy: {:.4f}'.format(correct / total)) 

test Accuracy: 0.6193


In [ ]:
predicted[:100] - 1

tensor([5, 5, 5, 2, 1, 5, 5, 5, 3, 5, 2, 3, 5, 3, 1, 5, 2, 3, 5, 0, 1, 5, 2, 5,
        0, 5, 4, 4, 5, 0, 3, 5, 5, 5, 1, 0, 5, 0, 3, 5, 0, 3, 2, 4, 5, 2, 5, 3,
        2, 2, 0, 1, 1, 4, 5, 1, 0, 0, 0, 0, 4, 5, 5, 2, 5, 3, 4, 5, 5, 2, 5, 6,
        3, 5, 5, 5, 0, 1, 0, 5, 5, 0, 3, 0, 1, 5, 5, 3, 5, 4, 3, 3, 0, 3, 3, 1,
        5, 3, 4, 5], device='cuda:0')

In [ ]:
labels[:100] - 1

tensor([2, 5, 5, 2, 1, 1, 5, 5, 3, 2, 2, 3, 5, 3, 1, 5, 2, 5, 5, 0, 4, 5, 2, 5,
        0, 5, 0, 4, 5, 3, 3, 5, 5, 5, 5, 5, 4, 5, 3, 6, 0, 3, 1, 4, 5, 2, 5, 1,
        2, 2, 0, 1, 1, 5, 1, 0, 0, 5, 1, 0, 5, 5, 5, 2, 6, 3, 0, 5, 5, 1, 5, 6,
        1, 3, 6, 5, 3, 3, 0, 5, 5, 3, 3, 3, 1, 5, 5, 3, 5, 4, 3, 3, 0, 2, 0, 1,
        4, 3, 3, 5], device='cuda:0')

In [ ]:
torch.save(act2vec, 'A2V.pt')

In [ ]:
inv_token = dict()
for key in token.keys():
    try: 
        inv_token[key] = {v: k for k, v in token[key].items()}
    except:
        pass
inv_token

{'action_type': {0: 'unknown',
  1: '검색',
  2: '제품 목록',
  3: '제품 세부정보',
  4: '장바구니 추가',
  5: '장바구니 삭제',
  6: '결제 시도',
  7: '구매 완료',
  8: '구매 환불'},
 'biz_unit': {0: 'unknown', 1: 'A01', 2: 'A02', 3: 'A03'},
 'clnt_age': {0: 'unknown',
  1: '10',
  2: '20',
  3: '30',
  4: '40',
  5: '50',
  6: '60'},
 'clnt_gender': {0: 'unknown', 1: 'F', 2: 'M'},
 'dvc_ctg_nm': {0: 'unknown', 1: 'mobile_app', 2: 'mobile_web', 3: 'PC'},
 'trfc_src': {0: 'unknown',
  1: 'DIRECT',
  2: 'PUSH',
  3: 'PORTAL_1',
  4: 'WEBSITE',
  5: 'PORTAL_2',
  6: 'PORTAL_3'}}

In [ ]:
show_data = X_test[:10000]

embedded_list = [x.detach().cpu().numpy() 
                    for x in act2vec.embed(torch.LongTensor(show_data).to(device))]
# embedded_list = [pca(x) for x in embedded_list]
embedded_list = [np.concatenate([show_data[:, :, i].reshape(-1, 1), x.reshape(-1, embedding_dim)], axis=1) 
                    for i, x in enumerate(embedded_list)]
embedded_dfs = [pd.DataFrame(x, columns=[cat_features[i]]+list(range(embedding_dim))) 
                    for i, x in enumerate(embedded_list)]

In [ ]:
import plotly.express as px
from sklearn.metrics.pairwise import euclidean_distances

for i, feature in enumerate(cat_features):
  embedded = embedded_dfs[i]
  embedded = (embedded
              .sort_values(feature)
              .groupby(feature)
              .sample(n=1))
  embedded.iloc[:, 0] = embedded.iloc[:, 0].replace(inv_token[feature])
  display(embedded)
  embedded.to_csv('{}_embedded.csv'.format(feature), index=False)

  # 각 클래스 별 유클리디안 거리 행렬
  dist = euclidean_distances(embedded.iloc[:, 1:], embedded.iloc[:, 1:])
  dist = pd.DataFrame(dist, index=embedded[feature], columns=embedded[feature])
  dist.index.name = None
  display(dist)
  dist.to_csv('{}_distance.csv'.format(feature))

,action_type,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
23926,검색,0.678901,-0.055033,-0.036150,0.437051,0.660694,-0.442195,-1.758835,-0.042029,0.285232,...,0.647220,0.630505,-1.498704,-0.767423,-0.715234,0.839904,-0.261127,-0.596262,1.081688,1.217919
38137,제품 목록,0.760826,1.442446,0.287831,-0.600497,-1.574844,1.559332,0.263662,-0.190154,-0.601153,...,-0.855940,-0.331367,-0.298501,2.476359,2.864059,1.722831,-2.060527,-1.347272,-0.386152,1.728333
29870,제품 세부정보,-0.751267,-0.215309,1.649605,-1.173818,1.007023,-0.213900,-0.071291,-0.066915,-0.763410,...,-1.624735,0.498470,1.626071,0.900805,-1.360268,-0.293216,-0.178286,1.002137,0.284000,1.349402
72987,장바구니 추가,1.024190,-0.449666,-0.648030,-0.751041,-0.959083,-0.896058,-1.389311,0.385931,-0.219969,...,-0.042537,-0.840186,1.442798,-0.171706,0.067975,-3.304097,-0.002607,-0.906652,0.116704,-0.013989
59310,장바구니 삭제,0.702562,-0.157508,0.358326,0.427703,-0.555402,0.534450,0.171102,-1.308369,0.014425,...,0.622893,1.449767,1.681832,0.045272,-0.786223,1.926808,-0.720853,1.551569,-1.238630,-1.506053
25998,결제 시도,-2.130040,-0.875991,-0.567734,0.508233,0.587354,-0.615232,1.632199,1.957627,0.907575,...,-0.228991,1.023343,-0.167125,0.499733,-0.193285,0.105188,0.821045,0.992098,0.375892,1.705217
60248,구매 완료,0.819502,-1.057858,1.582993,-0.932959,-0.215033,-0.066929,0.041406,-1.428491,0.028866,...,-0.882871,-1.075325,-0.788809,-0.949873,-1.624396,1.119201,0.832822,-0.122426,-0.370472,-0.015444
49204,구매 환불,0.293075,-0.965719,-0.524684,-1.310791,-0.097800,-1.580823,0.197240,-1.836307,-0.892016,...,-0.659974,0.388736,-2.830507,0.601129,-0.520901,-0.003207,-0.757737,-1.078987,-0.117290,-1.388077


action_type,검색,제품 목록,제품 세부정보,장바구니 추가,장바구니 삭제,결제 시도,구매 완료,구매 환불
검색,0.000000,12.967846,11.890119,11.449130,12.810497,11.621417,11.795477,12.008439
제품 목록,12.967846,0.000000,13.941494,13.546814,13.024756,12.937592,12.628317,12.644801
제품 세부정보,11.890119,13.941494,0.000000,11.453558,12.745847,11.556586,12.369072,12.808283
장바구니 추가,11.449130,13.546814,11.453558,0.000000,13.233790,12.697524,12.381038,12.329327
장바구니 삭제,12.810497,13.024756,12.745847,13.233790,0.000000,12.517648,12.168433,11.060349
결제 시도,11.621417,12.937592,11.556586,12.697524,12.517648,0.000000,11.682380,12.309726
구매 완료,11.795477,12.628317,12.369072,12.381038,12.168433,11.682380,0.000000,10.368863
구매 환불,12.008439,12.644801,12.808283,12.329327,11.060349,12.309726,10.368863,0.000000


,biz_unit,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
72339,A01,-0.872954,-0.821223,-0.691758,-0.549028,0.542050,-0.380179,-0.484453,-0.779257,0.486943,...,0.337375,1.373110,0.525952,0.084767,-1.220497,0.331262,-0.196922,-0.674504,-1.087793,-0.543493
75658,A02,-0.274569,1.145520,-1.190786,1.398952,0.082088,1.406090,0.060699,0.394536,2.415792,...,-0.093622,1.137803,0.420236,-1.056776,-0.511725,-0.159132,-1.966218,-0.582458,-0.446544,0.257184
14678,A03,-0.882739,0.026257,0.350901,0.106827,-0.273052,0.747686,0.736377,1.309724,0.528156,...,2.962505,-1.393675,-0.413522,-1.417096,0.740290,0.141767,0.225955,0.579892,0.279098,1.733267


biz_unit,A01,A02,A03
A01,0.000000,11.416402,11.085856
A02,11.416402,0.000000,10.206296
A03,11.085856,10.206296,0.000000


,trfc_src,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
23100,unknown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
13998,DIRECT,-0.170050,-0.928764,-0.648405,-0.528803,0.022333,0.756734,0.635076,-0.408238,0.907960,...,0.011642,-3.629623,1.629461,1.105850,1.315560,0.968467,0.009368,-0.025770,0.922594,0.628928
18901,PUSH,-0.299724,-0.433853,0.483730,-1.742684,0.200636,-0.983549,-0.351789,-0.076745,1.731866,...,0.612456,0.635035,-0.948588,2.086246,0.547866,0.883124,1.105919,1.199458,0.799326,0.121521
88745,PORTAL_1,-0.242498,-0.657965,-2.799849,-0.044956,0.712253,-0.152565,0.213557,0.644276,-0.040465,...,-3.133237,0.875305,-1.134688,1.029642,0.036194,-1.196985,-0.522759,0.603382,0.021493,-0.378278
53469,WEBSITE,1.382747,0.176453,0.043194,1.547610,0.318252,0.662593,1.290956,-0.117417,0.479198,...,2.255981,-0.481841,0.724681,0.480754,-0.501281,0.022210,-0.836238,-1.404494,-1.197080,1.124339
87194,PORTAL_2,-0.568625,-1.318074,-1.091920,-1.560290,-0.950436,1.642408,1.205057,1.591452,-1.285209,...,-0.245055,0.927649,0.180537,-1.087505,-0.281995,-0.678087,-1.082954,1.803727,-0.208436,-0.218360
78305,PORTAL_3,1.351898,-0.473915,0.328453,-0.532400,0.816705,-0.135833,0.937762,0.873021,1.139976,...,0.005719,-0.603817,-1.396039,-0.510720,0.934910,0.111750,-2.121657,-0.906762,0.856916,0.203873


...

dvc_ctg_nm,unknown,mobile_app,mobile_web,PC
unknown,0.000000,7.891559,7.570361,8.373373
mobile_app,7.891559,0.000000,11.344673,10.310809
mobile_web,7.570361,11.344673,0.000000,11.958561
PC,8.373373,10.310809,11.958561,0.000000


,clnt_gender,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
764,unknown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6371,F,0.899682,-0.481289,0.257508,-0.731883,0.545823,1.227445,-1.016289,-0.567574,-0.442408,...,-0.088577,0.394136,0.179437,1.694499,-1.205889,-1.087401,0.038427,0.390773,-1.124691,0.772314
99728,M,0.294238,-1.988044,0.252468,0.160835,-0.761110,-0.455324,-1.190260,-0.219093,-1.274325,...,-0.225357,-0.117048,0.056384,0.697837,0.371430,-1.277486,0.196144,-0.162677,-3.234701,0.350061


clnt_gender,unknown,F,M
unknown,0.000000,7.026478,7.782161
F,7.026478,0.000000,10.072312
M,7.782161,10.072312,0.000000


,clnt_age,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
51147,unknown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
71089,10,0.195645,0.152880,-0.559084,1.577626,0.251080,0.970962,-0.625481,0.634736,2.374664,...,1.345295,0.997315,0.997516,-0.800822,0.536696,-0.433419,-1.449767,-1.373459,-0.182353,-0.890084
24835,20,0.174954,-1.561000,1.184435,-0.819845,0.677644,-1.209814,-0.567188,-1.034869,0.554766,...,0.528737,-0.457135,0.692194,1.506658,0.082598,-0.606576,1.968546,-0.351485,-0.972599,0.167735
68114,30,0.259025,1.359004,-1.384163,-1.073244,0.136911,-0.097718,0.640525,-0.769206,-0.243898,...,-1.449098,-1.420436,1.233091,1.476496,0.313701,-0.240543,-1.369334,0.953891,-1.123012,1.810884
42468,40,0.450258,1.602729,-0.326546,-0.131983,0.261307,0.186458,0.635000,0.632085,-0.460823,...,-0.118350,-0.179283,-0.374140,0.114262,-1.541171,-1.442407,1.037462,0.214660,0.429848,-1.087166
4850,50,0.350536,-0.119101,-0.439464,-2.093225,0.853890,-1.389823,-1.120395,0.217681,0.477376,...,0.250859,-0.317195,-0.251555,-1.068368,0.000726,-0.873761,-0.441523,0.274513,-1.604717,0.046583
98028,60,1.298735,1.904997,-0.169302,0.279795,-0.018396,-0.285664,-1.043321,-1.486076,1.508888,...,1.173318,1.909290,0.142377,-0.113950,-0.705366,0.894749,-0.570800,0.280905,-1.267946,0.015383


clnt_age,unknown,10,20,30,40,50,60
unknown,0.000000,8.053244,8.349039,7.552833,8.654291,6.606078,8.670784
10,8.053244,0.000000,11.148755,11.429549,12.548259,10.533346,10.473566
20,8.349039,11.148755,0.000000,11.610531,13.116719,9.830203,11.505132
30,7.552833,11.429549,11.610531,0.000000,11.316377,9.371757,11.201501
40,8.654291,12.548259,13.116719,11.316377,0.000000,10.280621,12.437638
50,6.606078,10.533346,9.830203,9.371757,10.280621,0.000000,11.608038
60,8.670784,10.473566,11.505132,11.201501,12.437638,11.608038,0.000000


In [ ]:
# PCA to plot vectors
from sklearn.decomposition import PCA

def pca(x):
    x = x.reshape(-1, embedding_dim)
    model = PCA(n_components=3)
    x = model.fit_transform(x)

    return x

embedded_list = [x.detach().cpu().numpy() 
                    for x in act2vec.embed(torch.LongTensor(show_data).to(device))]
embedded_list = [pca(x) for x in embedded_list]
embedded_list = [np.concatenate([show_data[:, :, i].reshape(-1, 1), x], axis=1) 
                    for i, x in enumerate(embedded_list)]
embedded_dfs = [pd.DataFrame(x, columns=[cat_features[i]]+list(range(3))) 
                    for i, x in enumerate(embedded_list)]

In [ ]:
for i, feature in enumerate(cat_features):
  embedded = embedded_dfs[i]
  embedded = (embedded
              .sort_values(feature)
              .groupby(feature)
              .sample(n=1))
  embedded.iloc[:, 0] = embedded.iloc[:, 0].replace(inv_token[feature])

  fig = px.scatter_3d(embedded, 
                  x=0, y=1, z=2,
                  color=embedded.iloc[:, 0],
                  title=embedded.columns[0])
  fig.update_layout(
      margin=dict(l=20, r=20, t=20, b=20)
  )
  fig.write_html("{}_plot.html".format(feature))
  fig.show()

1) 4: 장바구니 삭제 & 7: 구매 환불은 서로 거리가 가깝다.  
2) 6: 구매 완료와 가까운 것들은 2: 제품 세부정보, 3: 장바구니 추가가 있다.  
   또한 7: 구매 환불과도 가까운데, 구매를 해야 환불도 가능하니 당연한 것.  
3) 뭐가 더 있을까용